In [ ]:
#Libraries
import sys
import argparse
import yaml

import cv2
import torch
from torch.autograd import Variable
from torchvision.models import resnet50, ResNet50_Weights

from models.yolov3 import *
from utils.utils import *
from utils.parse_yolo_weights import parse_yolo_weights
from utils.vis_bbox import vis_bbox

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Choose config file for this demo
cfg_path = '/home/samsung/Downloads/PyTorch_Gaussian_YOLOv3-master/config/gaussian_yolov3_eval.cfg'

# Specify checkpoint file which contains the weights of the model you want to use
ckpt_path = '/home/samsung/Downloads/PyTorch_Gaussian_YOLOv3-master/gaussian_yolov3_coco.pth' 
ckpt = '/home/samsung/Downloads/PyTorch_Gaussian_YOLOv3-master/resnet50-11ad3fa6.pth'

# Path to the image file fo the demo
image_path = '/home/samsung/Downloads/PyTorch_Gaussian_YOLOv3-master/data/gaussian_yolov3/traffic_1.jpg'

# Detection threshold
detect_thresh = 0.3

# Use CPU if gpu < 0 else use GPU
gpu = -1

In [ ]:
# Load configration parameters for this demo
with open(cfg_path, 'r') as f:
    cfg = yaml.load(f)

model_config = cfg['MODEL']
imgsize = cfg['TEST']['IMGSIZE']
confthre = cfg['TEST']['CONFTHRE'] 
nmsthre = cfg['TEST']['NMSTHRE']
gaussian = cfg['MODEL']['GAUSSIAN']

# if detect_thresh is not specified, the parameter defined in config file is used
if detect_thresh:
    confthre = detect_thresh

In [ ]:
# Load model
model = YOLOv3(model_config)

# Load weight from the checkpoint
#print("loading checkpoint %s" % (ckpt_path))
state = torch.load(ckpt)

#Add all weights to an empty dict
md_dict = model.state_dict()

pretrained_dict={}
#for k,v in state.items():
#    pretrained_dict[k]=v


checkpoint= torch.load(ckpt_path)
for k,v in checkpoint.items():
    pretrained_dict[k]=v
md_dict={k:v for k, v in md_dict.items() if k in pretrained_dict}
md_dict.update(pretrained_dict)

#Load ImageNet and YOLO weights to model
model.load_state_dict(md_dict)

model.train()
model.eval()

if gpu >= 0:
    # Send model to GPU
    model.cuda()

In [ ]:
# Load image
img = cv2.imread(image_path)

# Preprocess image
img_raw = img.copy()[:, :, ::-1].transpose((2, 0, 1))
img, info_img = preprocess(img, imgsize, jitter=0)  # info = (h, w, nh, nw, dx, dy)
img = np.transpose(img / 255., (2, 0, 1))
img = torch.from_numpy(img).float().unsqueeze(0)

if gpu >= 0:
    # Send model to GPU
    img = Variable(img.type(torch.cuda.FloatTensor))
else:
    img = Variable(img.type(torch.FloatTensor))

In [ ]:
# Inference
with torch.no_grad():
    outputs = model(img)
    outputs = postprocess(outputs, 80, confthre, nmsthre)

if outputs[0] is None:
    #Exit out of code with no errors
    print("No Objects Deteted!!")
    sys.exit(0)

# Visualize detected bboxes
coco_class_names, coco_class_ids, coco_class_colors = get_coco_label_names()

bboxes = list()
classes = list()
scores = list()
colors = list()
sigmas = list()

for output in outputs[0]:
    x1, y1, x2, y2, conf, cls_conf, cls_pred = output[:7]
    if gaussian:
        sigma_x, sigma_y, sigma_w, sigma_h = output[7:]
        sigmas.append([sigma_x, sigma_y, sigma_w, sigma_h])

    cls_id = coco_class_ids[int(cls_pred)]
    box = yolobox2label([y1, x1, y2, x2], info_img)
    bboxes.append(box)
    classes.append(cls_id)
    scores.append(cls_conf * conf)
    colors.append(coco_class_colors[int(cls_pred)])
    
    # image size scale used for sigma visualization
    h, w, nh, nw, _, _ = info_img
    sigma_scale_img = (w / nw, h / nh)
    
fig, ax = vis_bbox(
    img_raw, bboxes, label=classes, score=scores, label_names=coco_class_names, sigma=sigmas, 
    sigma_scale_img=sigma_scale_img,
    sigma_scale_xy=2., sigma_scale_wh=2.,  # 2-sigma
    show_inner_bound=False,  # do not show inner rectangle for simplicity
    instance_colors=colors, linewidth=3)

fig.savefig('demo.png')